In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Разархивируем датасет для обучения**






In [ ]:
!unzip drive/MyDrive/collab/train.zip -d dataset/

In [ ]:
!pip install segmentation-models

**Импопрт библиотек**

In [ ]:
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import pandas as pd
import cv2
from scipy import ndimage
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import ModelCheckpoint

**Функция декодирования маски и генератор**

In [ ]:
def rle_decode(mask_rle, shape=(640, 480, 1)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1

    img = img.reshape(shape)

    return img


def keras_generator(gen_df, batch_size):
    while True:
        x_batch = []
        y_batch = []

        for i in range(batch_size):
            img_name, mask_rle = gen_df.sample(1).values[0]
            img = load_img(f'{img_name}', color_mode='rgba')#, target_size=(480, 640))
            img = img.convert('RGB')
            img = img_to_array(img)
            mask = rle_decode(mask_rle)
            mask = ndimage.rotate(mask, -90)
            mask = cv2.flip(mask, 1)

            x_batch += [img]
            y_batch += [mask]

        x_batch = np.array(x_batch) / 255.
        y_batch = np.array(y_batch)

        yield x_batch, np.expand_dims(y_batch, -1)


**Импорт датасета и разделение валидацию**

In [ ]:
data = pd.read_csv('dataset/train/segmentation.csv')
train_data = data[:17621]
val_data = data[17621:]


**Создание модели и колбэка**

In [ ]:
model = sm.Unet('vgg19', encoder_weights='imagenet', input_shape=(480, 640 ,3))
model.summary()


best_w = keras.callbacks.ModelCheckpoint("drive/MyDrive/collab/segmetn1e_{epoch}", monitor='val_accuracy', verbose=1, 
                             save_best_only=True)


model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              #optimizer = 'adam',
              metrics=['accuracy'])


**Обучение модели**

---


мне понадобилось более 100 эпох

In [ ]:
model.fit(
    keras_generator(train_data, batch_size=8),
    steps_per_epoch=100,
    callbacks=best_w,
    validation_data=keras_generator(val_data, batch_size=8),
    validation_steps=50,
    epochs=100
)


**Загрузка модели и обучение загруженной модели**

---
если время в калабе кончилось, а результат вам не нравится


In [ ]:
model_loud = keras.models.load_model('drive/MyDrive/collab/segmetn1e_55_V88__6')

model_loud.fit(
    keras_generator(train_data, batch_size=8),
    steps_per_epoch=100,
    callbacks=best_w,
    validation_data=keras_generator(val_data, batch_size=8),
    validation_steps=50,
    epochs=100
)


**Разархивируем датасет для сабмита**

In [ ]:
!unzip drive/MyDrive/collab/test.zip -d dataset/

**Функция кодирования массива в маску**

In [ ]:
def rle_encoding(x):
    '''
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = np.where(x.T.flatten()==1)[0] # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths
  

***Функция декодирования маски в массив***

In [ ]:
def rle_decode(mask_rle, shape=(640, 480, 1)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1

    img = img.reshape(shape)

    return img


**Загрузка обученной модели для сабмита**

In [ ]:
model_loud = keras.models.load_model('drive/MyDrive/collab/segmetn1e_55_V88__6')


**Предикт**

In [ ]:
data = pd.read_csv('drive/MyDrive/collab/sample_submission.csv')

for x in range(5206):
  path = data['image'][x]
  img = load_img(f'{path}', color_mode='rgba')#, target_size=(480, 640))
  img = img.convert('RGB')
  img = img_to_array(img)
  img = img * 1/255
  img = np.expand_dims(img, axis=0)

  mask = model_loud.predict(img)

  mask = mask.reshape(480,640)
  mask = np.rint(mask)

  mask_rle = rle_encoding(mask)

  rles.append(mask_rle)


**Перевод чисел в строку и запись в файл**

In [ ]:
allrles = rles


for x in range(len(allrles)):
  for i in range(len(allrles[x])):
    allrles[x][i] = str(allrles[x][i])


new_lst=[]
new_ln = None

for x in range(len(allrles)):
  new_ln = " ".join(allrles[x])
  new_lst.append(new_ln)



for x in range(len(allrles)):
  data['segmentation'][x] = new_lst[x]


data.to_csv('drive/MyDrive/collab/predict.csv', index=False)
